# packages and code

In [1]:
from tps import ThinPlateSpline
from tps_scripts import *

# Paths

In [2]:
# paths

root_dir = os.path.abspath('../')

mri_dir = os.path.abspath('../mri/results')
path_to_json = os.path.join(mri_dir, 'orientation.mrk.json')

xml_path = os.path.join(root_dir,"templates/markers_and_bone_markers_in_bodies.xml")
osim_path = os.path.join(root_dir,'templates/RajagopalModified_generic_copy.osim')

working_dir = os.getcwd()
vtp_path = os.path.join(working_dir, 'Geometry')

In [ ]:
# set output path -- this file will save results after the MRI adjustments of muscles
output_path = os.path.join(root_dir, 'final_results')
if not os.path.exists(output_path):
    print(output_path)
    os.makedirs(output_path)

# set path for controls
control_path = os.path.join(root_dir, 'mri', 'control')
if not os.path.exists(control_path):
    print(control_path)
    os.makedirs(control_path)

c:\Users\User\OneDrive\Documents\WORK\JRF_GaitAnalysis\modelling_paper\repo\folder\final_results


In [4]:
print(control_path)

c:\Users\User\OneDrive\Documents\WORK\JRF_GaitAnalysis\modelling_paper\repo\folder\mri\control


# load osim

In [5]:
# OSIM
osim_bone_markers = OsimBoneMarkers(xml_path)
osim_bone_markers_df = osim_bone_markers.data_frame()

In [6]:
osim_muscles_and_wrapping = OsimMusclePathsAndWrapping(osim_path)

osim_muslce_paths_df = osim_muscles_and_wrapping.df_muscles

wrapping_surfaces_df = osim_muscles_and_wrapping.wrp_df

# load mri and match to osim

In [7]:
# MRI
mri_bone_markers = MRIBoneMarkers(path_to_json)
mri_bone_markers_df = mri_bone_markers.json_to_df()

# collections by bodies

In [8]:
osim_mri_bone_by_bodies = OsimMriBoneByBodies(osim_bone_markers_df, mri_bone_markers_df)
osim_skin_markers_dict = osim_mri_bone_by_bodies.split_osim_skin_by_bodies()

osim_dict = osim_mri_bone_by_bodies.split_osim_by_bodies()
mri_dict = osim_mri_bone_by_bodies.split_mri_by_osim_bodies()

osim_muscls_by_bodies = OsimMusclesByBodies(osim_muslce_paths_df)
osim_muscls_dict = osim_muscls_by_bodies.extract_all()

osim_wraps_by_bodies = OsimWrapsByBodies(wrapping_surfaces_df)
osim_wraps_dict = osim_wraps_by_bodies.wraps_to_points_by_bodies()

# pelvis tps

In [9]:
# exclude markers to control for the quality of the spline
exclude = ['sacroiliac_r', 'sacroiliac_l']#,'5th_sacr_v'

# create the spline
pelvis_tps = OneBodyTPS('pelvis', osim_dict['pelvis'], mri_dict['pelvis'], 
                        osim_skin = osim_skin_markers_dict['pelvis'],
                        osim_muscle = osim_muscls_dict['pelvis'], 
                        exclude_bone_markers = exclude)

# transform all objects: bone markers are transformed and should fit the chosen subset save for the exclusions
osim_bone = osim_dict['pelvis']
pelvis_transformed_bone = pelvis_tps.transformed_bone
pelvis_transformed_muscle = pelvis_tps.transformed_muscle
pelvis_transformed_wraps = pelvis_tps.apply_tps_to_wraps(osim_wraps_dict['pelvis'])
pelvis_transformed_skin = pelvis_tps.transformed_skin

l_pelvis = ImportVTP(vtp_path, 'l_pelvis.vtp')
r_pelvis = ImportVTP(vtp_path, 'r_pelvis.vtp')
sacrum = ImportVTP(vtp_path, 'sacrum.vtp')

l_pelvis_transformed = pv.PolyData(pelvis_tps.apply_spline_to_surface(l_pelvis.mesh))
r_pelvis_transformed = pv.PolyData(pelvis_tps.apply_spline_to_surface(r_pelvis.mesh))
sacrum_transformed = pv.PolyData(pelvis_tps.apply_spline_to_surface(sacrum.mesh))

# plot resutls
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_pelvis.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(r_pelvis.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(sacrum.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(l_pelvis_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_pelvis_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(sacrum_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pelvis_transformed_bone),  color='blue', point_size=10.0)
if len(exclude) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['pelvis'].loc[exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x246698bc150_0&reconnect=auto" class="pyvis…

In [10]:
# import adjusted muscles and warping surfaces after control in Slicer
if os.path.isfile(os.path.join(control_path, 'new_muscles.csv')) and os.path.isfile(os.path.join(control_path, 'new_wraps.csv')):
    print('yes')
    muscles = pd.read_csv(os.path.join(control_path, 'new_muscles.csv'),index_col='name')
    wraps = pd.read_csv(os.path.join(control_path, 'new_wraps.csv'),index_col='name')

    pelv_m = muscles[muscles['body'] == 'pelvis']
    pelv_w = wraps[wraps['body'] == 'pelvis']

    pelvis_transformed_muscle = np.asarray(pelv_m[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')
    pelvis_transformed_wraps[2] = np.asarray(pelv_w[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')

    

yes


In [11]:
# rotate results to child frame
bone_markers = list(mri_dict['pelvis'].index)
pelvis_axes = GetPelvisAxes(pelvis_transformed_bone, bone_markers)

pelvis_bone_in_child = pelvis_axes.bone_transformed
pelvis_skin_in_child = pelvis_axes.apply_to_non_bone(pelvis_transformed_skin)
pelvis_muscles_in_child = pelvis_axes.apply_to_non_bone(pelvis_transformed_muscle)
pelvis_wrap_translations_in_child = pelvis_axes.apply_to_non_bone(pelvis_transformed_wraps[2])

l_pelvis_surface_in_child = pv.PolyData(pelvis_axes.apply_to_non_bone(l_pelvis_transformed.points), l_pelvis_transformed.faces)
r_pelvis_surface_in_child = pv.PolyData(pelvis_axes.apply_to_non_bone(r_pelvis_transformed.points), r_pelvis_transformed.faces)
sacrum_surface_in_child = pv.PolyData(pelvis_axes.apply_to_non_bone(sacrum_transformed.points), sacrum_transformed.faces)

In [12]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_pelvis.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(r_pelvis.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(sacrum.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(osim_bone.to_numpy()),  color='blue', point_size=10.0) 
actor = plotter.add_mesh(pv.PolyData(osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['pelvis'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['pelvis']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(l_pelvis_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_pelvis_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(sacrum_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pelvis_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(pelvis_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(pelvis_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x2466ba9e910_1&reconnect=auto" class="pyvis…

In [14]:
# record results
surfaces = [l_pelvis_surface_in_child, r_pelvis_surface_in_child, sacrum_surface_in_child]
surface_names = ['l_pelvis', 'r_pelvis', 'sacrum']

bone_markers = pelvis_bone_in_child
bone_marker_names = list(mri_dict['pelvis'].index)

skin_markers = pelvis_skin_in_child
skin_markers_names = list(osim_skin_markers_dict['pelvis'].index)

muscle_markers = pelvis_muscles_in_child
muscle_marker_names = list(osim_muscls_dict['pelvis'].index)

wrap_translations = pelvis_wrap_translations_in_child
wrap_names = list(osim_wraps_dict['pelvis']['name'])

pelvis_results = ScaleAndRecordData('pelvis', output_path,
                                    transformed_bone_markers = bone_markers, bone_marker_names = bone_marker_names,
                                    transformed_skin_markers = skin_markers, skin_marker_names = skin_markers_names,
                                    transformed_muscle_markers = muscle_markers, muscle_marker_names = muscle_marker_names,
                                    transformed_wrap_translations = wrap_translations, wrap_names = wrap_names,
                                    list_transformed_surfaces_as_PolyData = surfaces, list_surface_names = surface_names )

# femur_r  tps

In [15]:
# exclude markers to control for the quality of the spline
to_exclude_femur_r = [] #'knee_r_center_in_femur_r','gr_troch_as_r', 'femur_r_poster_diaph_25', 'femur_r_anter_diaph_25', 

# calculate the spline
femur_r_tps = OneBodyTPS('femur_r', osim_dict['femur_r'], 
                         mri_dict['femur_r'], 
                        osim_muscle = osim_muscls_dict['femur_r'], 
                         osim_skin = osim_skin_markers_dict['femur_r'],
                         exclude_bone_markers = to_exclude_femur_r)

# transform all objects
femur_r_osim_bone = osim_dict['femur_r']
femur_r_transformed_bone = femur_r_tps.transformed_bone
femur_r_transformed_skin = femur_r_tps.transformed_skin
femur_r_transformed_muscle = femur_r_tps.transformed_muscle
femur_r_transformed_wraps = femur_r_tps.apply_tps_to_wraps(osim_wraps_dict['femur_r'])

r_femur = ImportVTP(vtp_path, 'r_femur.stl')
femur_r_transformed = femur_r_tps.apply_spline_to_surface(r_femur.mesh)

# plot results
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_transformed_bone),  color='blue', point_size=10.0)
if len(to_exclude_femur_r) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['femur_r'].loc[to_exclude_femur_r].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')


Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x24642c902d0_2&reconnect=auto" class="pyvis…

In [16]:
# import adjusted muscles and warping surfaces after control in Slicer
if os.path.isfile(os.path.join(control_path, 'new_muscles.csv')) and os.path.isfile(os.path.join(control_path, 'new_wraps.csv')):
    muscles = pd.read_csv(os.path.join(control_path, 'new_muscles.csv'),index_col='name')
    wraps = pd.read_csv(os.path.join(control_path, 'new_wraps.csv'),index_col='name')

    femur_r_m = muscles[muscles['body'] == 'femur_r']
    femur_r_w = wraps[wraps['body'] == 'femur_r']

    femur_r_transformed_muscle = np.asarray(femur_r_m[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')
    femur_r_transformed_wraps[2] = np.asarray(femur_r_w[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')


In [17]:
# create rotations to child
bone_markers = list(mri_dict['femur_r'].index)
femur_r_axes = GetFemurAxes(femur_r_transformed_bone,
                            bone_markers, 
                            femur_skin_numpy = femur_r_transformed_skin,
                            femur_muscles_numpy = femur_r_transformed_muscle,
                            femur_wraps_numpy = femur_r_transformed_wraps[2],
                            femur_surface_numpy = femur_r_transformed.points)
# rotate to child
femur_r_bone_in_child = femur_r_axes.bone_transformed
femur_r_skin_in_child = femur_r_axes.femur_skin
femur_r_muscles_in_child = femur_r_axes.femur_muscles
femur_r_wrap_translations_in_child = femur_r_axes.femur_wraps
femur_r_surface_in_child = pv.PolyData(femur_r_axes.femur_surface, femur_r_transformed.faces)

the determinant is less than zero, recalculate r
the determinant is less than zero, recalculate r


In [18]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['femur_r'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['femur_r']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(femur_r_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(femur_r_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x24642d6aad0_3&reconnect=auto" class="pyvis…

In [19]:

# # record results
surfaces = [femur_r_surface_in_child]
surface_names = ['femur_r']
bone_markers = femur_r_bone_in_child
bone_marker_names = list(mri_dict['femur_r'].index)

skin_markers = femur_r_skin_in_child
skin_marker_names = list(osim_skin_markers_dict['femur_r'].index)

muscle_markers = femur_r_muscles_in_child
muscle_marker_names = list(osim_muscls_dict['femur_r'].index)
wrap_translations = femur_r_wrap_translations_in_child
wrap_names = list(osim_wraps_dict['femur_r']['name'])

femur_r_results = ScaleAndRecordData('femur_r', output_path,
                                    transformed_bone_markers = bone_markers, bone_marker_names = bone_marker_names,
                                    transformed_skin_markers = skin_markers, skin_marker_names = skin_marker_names,
                                    transformed_muscle_markers = muscle_markers, muscle_marker_names = muscle_marker_names,
                                    transformed_wrap_translations = wrap_translations, wrap_names = wrap_names,
                                    list_transformed_surfaces_as_PolyData = surfaces, list_surface_names = surface_names)

# patella_r tps

In [20]:
# exclude markers
patella_r_exclude =  None

# calculate spline
patella_r_tps = OneBodyTPS('patella_r', osim_dict['patella_r'], 
                         mri_dict['patella_r'], 
                         osim_muscle = osim_muscls_dict['patella_r'], 
                         osim_skin = None, 
                         exclude_bone_markers = patella_r_exclude)

# transform objects
patella_r_osim_bone = osim_dict['patella_r']
patella_r_transformed_bone = patella_r_tps.transformed_bone
patella_r_transformed_muscle = patella_r_tps.transformed_muscle

r_patella = ImportVTP(vtp_path, 'r_patella.vtp')

patella_r_transformed = patella_r_tps.apply_spline_to_surface(r_patella.mesh)


# plot results
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_transformed_bone),  color='blue', point_size=10.0)
#if len(patella_r_exclude) >= 1:
#actor = plotter.add_mesh(pv.PolyData(osim_mri_bone_by_bodies.patella_r_mri.loc[patella_r_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x24642e48410_4&reconnect=auto" class="pyvis…

In [21]:
# import adjusted muscles and warping surfaces after control in Slicer
if os.path.isfile(os.path.join(control_path, 'new_muscles.csv')) and os.path.isfile(os.path.join(control_path, 'new_wraps.csv')):
    muscles = pd.read_csv(os.path.join(control_path, 'new_muscles.csv'),index_col='name')
    wraps = pd.read_csv(os.path.join(control_path, 'new_wraps.csv'),index_col='name')

    patella_r_m = muscles[muscles['body'] == 'patella_r']
    patella_r_transformed_muscle = np.asarray(patella_r_m[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')

In [22]:
patella_bone_markers = list(mri_dict['patella_r'].index)

patella_rotate = femur_r_axes.transform_patella(patella_r_transformed_bone, 
                                                  patella_bone_markers,
                                                  patella_muscles_numpy = patella_r_transformed_muscle,  
                                                  patella_surface_numpy = patella_r_transformed.points)

patella_r_bone_in_child = patella_rotate[0]
patella_r_muscle_in_child = patella_rotate[1] 
patella_r_surface_in_child = pv.PolyData(patella_rotate[2], patella_r_transformed.faces)

In [23]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(patella_r_muscle_in_child),  color='black', point_size=5.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x246377df210_5&reconnect=auto" class="pyvis…

In [24]:
# # record results
surfaces = [patella_r_surface_in_child]
surface_names = ['patella_r']
bone_markers = patella_r_bone_in_child
bone_marker_names = list(mri_dict['patella_r'].index)
muscle_markers = patella_r_muscle_in_child
muscle_marker_names = list(osim_muscls_dict['patella_r'].index)
wrap_translations = []
wrap_names = []
femur_r_results = ScaleAndRecordData('patella_r', output_path,
                                    transformed_bone_markers = bone_markers, bone_marker_names = bone_marker_names,
                                    transformed_muscle_markers = muscle_markers, muscle_marker_names = muscle_marker_names,
                                    transformed_wrap_translations = wrap_translations, wrap_names = wrap_names,
                                    list_transformed_surfaces_as_PolyData = surfaces, list_surface_names = surface_names)

# femur_l  tps

In [25]:
# exclude markers to control for the quality of the spline
to_exclude_femur_l = [] #, 'gr_troch_ps_l','gr_troch_lat_l', 'gr_troch_as_l'

# calculate the spline
femur_l_tps = OneBodyTPS('femur_l', osim_dict['femur_l'], 
                         mri_dict['femur_l'], 
                         osim_skin = osim_skin_markers_dict['femur_l'],
                         osim_muscle = osim_muscls_dict['femur_l'], 
                         exclude_bone_markers = to_exclude_femur_l)

# transform all objects
femur_l_osim_bone = osim_dict['femur_l']
femur_l_transformed_bone = femur_l_tps.transformed_bone
femur_l_transformed_skin = femur_l_tps.transformed_skin
femur_l_transformed_muscle = femur_l_tps.transformed_muscle
femur_l_transformed_wraps = femur_l_tps.apply_tps_to_wraps(osim_wraps_dict['femur_l'])

l_femur = ImportVTP(vtp_path, 'l_femur.stl')
femur_l_transformed = femur_l_tps.apply_spline_to_surface(l_femur.mesh)

# plot results
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_transformed_bone),  color='blue', point_size=10.0)
if len(to_exclude_femur_l) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['femur_l'].loc[to_exclude_femur_l].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x24673c2b610_6&reconnect=auto" class="pyvis…

In [26]:
# import adjusted muscles and warping surfaces after control in Slicer
if os.path.isfile(os.path.join(control_path, 'new_muscles.csv')) and os.path.isfile(os.path.join(control_path, 'new_wraps.csv')):
    muscles = pd.read_csv(os.path.join(control_path, 'new_muscles.csv'),index_col='name')
    wraps = pd.read_csv(os.path.join(control_path, 'new_wraps.csv'),index_col='name')

    femur_l_m = muscles[muscles['body'] == 'femur_l']
    femur_l_w = wraps[wraps['body'] == 'femur_l']

    femur_l_transformed_muscle = np.asarray(femur_l_m[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')
    femur_l_transformed_wraps[2] = np.asarray(femur_l_w[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')

In [27]:
# create rotations to child
bone_markers = list(mri_dict['femur_l'].index)
femur_l_axes = GetFemurAxes(femur_l_transformed_bone,
                            bone_markers, 
                            femur_skin_numpy = femur_l_transformed_skin,
                            femur_muscles_numpy = femur_l_transformed_muscle,
                            femur_wraps_numpy = femur_l_transformed_wraps[2],
                            femur_surface_numpy = femur_l_transformed.points)
# rotate to child
femur_l_bone_in_child = femur_l_axes.bone_transformed
femur_l_skin_in_child = femur_l_axes.femur_skin
femur_l_muscles_in_child = femur_l_axes.femur_muscles
femur_l_wrap_translations_in_child = femur_l_axes.femur_wraps
femur_l_surface_in_child = pv.PolyData(femur_l_axes.femur_surface, femur_l_transformed.faces)

the determinant is less than zero, recalculate r
the determinant is less than zero, recalculate r


In [28]:
# plot results of the rotation
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['femur_l'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['femur_l']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(femur_l_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(femur_l_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x24687c33590_7&reconnect=auto" class="pyvis…

In [29]:

# record results
surfaces = [femur_l_surface_in_child]
surface_names = ['femur_l']
bone_markers = femur_l_bone_in_child
bone_marker_names = list(mri_dict['femur_l'].index)

skin_markers = femur_l_skin_in_child
skin_marker_names = list(osim_skin_markers_dict['femur_l'].index)

muscle_markers = femur_l_muscles_in_child
muscle_marker_names = list(osim_muscls_dict['femur_l'].index)
wrap_translations = femur_l_wrap_translations_in_child
wrap_names = list(osim_wraps_dict['femur_l']['name'])

femur_r_results = ScaleAndRecordData('femur_l', output_path,
                                    transformed_bone_markers = bone_markers, bone_marker_names = bone_marker_names,
                                    transformed_skin_markers = skin_markers, skin_marker_names = skin_marker_names,
                                    transformed_muscle_markers = muscle_markers, muscle_marker_names = muscle_marker_names,
                                    transformed_wrap_translations = wrap_translations, wrap_names = wrap_names,
                                    list_transformed_surfaces_as_PolyData = surfaces, list_surface_names = surface_names)

# patella_l  tps

In [30]:
patella_l_exclude =  None

patella_l_tps = OneBodyTPS('patella_l', osim_dict['patella_l'], 
                         mri_dict['patella_l'], 
                         osim_muscle = osim_muscls_dict['patella_l'], 
                         osim_skin = None,
                         exclude_bone_markers = patella_l_exclude)

patella_l_osim_bone = osim_dict['patella_l']
patella_l_transformed_bone = patella_l_tps.transformed_bone
patella_l_transformed_muscle = patella_l_tps.transformed_muscle

l_patella = ImportVTP(vtp_path, 'l_patella.vtp')

patella_l_transformed = patella_l_tps.apply_spline_to_surface(l_patella.mesh)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_transformed_bone),  color='blue', point_size=10.0)
# if len(patella_r_exclude) >= 1:
#actor = plotter.add_mesh(pv.PolyData(osim_mri_bone_by_bodies.patella_r_mri.loc[patella_r_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x24687c62fd0_8&reconnect=auto" class="pyvis…

In [31]:
# import adjusted muscles and warping surfaces after control in Slicer
if os.path.isfile(os.path.join(control_path, 'new_muscles.csv')) and os.path.isfile(os.path.join(control_path, 'new_wraps.csv')):
    muscles = pd.read_csv(os.path.join(control_path, 'new_muscles.csv'),index_col='name')
    wraps = pd.read_csv(os.path.join(control_path, 'new_wraps.csv'),index_col='name')

    patella_l_m = muscles[muscles['body'] == 'patella_l']
    patella_l_transformed_muscle = np.asarray(patella_l_m[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')

In [32]:
patella_bone_markers = list(mri_dict['patella_l'].index)

patella_rotate = femur_l_axes.transform_patella(patella_l_transformed_bone, 
                                                  patella_bone_markers,
                                                  patella_muscles_numpy = patella_l_transformed_muscle,  
                                                  patella_surface_numpy = patella_l_transformed.points)

patella_l_bone_in_child = patella_rotate[0]
patella_l_muscle_in_child = patella_rotate[1] 
patella_l_surface_in_child = pv.PolyData(patella_rotate[2], patella_l_transformed.faces)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(patella_l_muscle_in_child),  color='black', point_size=5.0)
plotter.show(jupyter_backend='trame')

the determinant is less than zero, recalculate r


Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x24673bf8b90_9&reconnect=auto" class="pyvis…

In [33]:
# record results
surfaces = [patella_l_surface_in_child]
surface_names = ['patella_l']
bone_markers = patella_l_bone_in_child
bone_marker_names = list(mri_dict['patella_l'].index)
muscle_markers = patella_l_muscle_in_child
muscle_marker_names = list(osim_muscls_dict['patella_l'].index)
wrap_translations = []
wrap_names = []
femur_r_results = ScaleAndRecordData('patella_l', output_path,
                                    transformed_bone_markers = bone_markers, bone_marker_names = bone_marker_names,
                                    transformed_muscle_markers = muscle_markers, muscle_marker_names = muscle_marker_names,
                                    transformed_wrap_translations = wrap_translations, wrap_names = wrap_names,
                                    list_transformed_surfaces_as_PolyData = surfaces, list_surface_names = surface_names)

# tibia_r tps

In [34]:
tibia_r_exclude =  [] #'tibia_r_anter_diaph_75',

tibia_r_tps = OneBodyTPS('tibia_r', osim_dict['tibia_r'], 
                         mri_dict['tibia_r'], 
                         osim_skin = osim_skin_markers_dict['tibia_r'],
                         osim_muscle = osim_muscls_dict['tibia_r'], 
                         exclude_bone_markers = tibia_r_exclude)

tibia_r_osim_bone = osim_dict['tibia_r']
tibia_r_transformed_bone = tibia_r_tps.transformed_bone
tibia_r_transformed_skin = tibia_r_tps.transformed_skin
tibia_r_transformed_muscle = tibia_r_tps.transformed_muscle
tibia_r_transformed_wraps = tibia_r_tps.apply_tps_to_wraps(osim_wraps_dict['tibia_r'])

r_tibia = ImportVTP(vtp_path, 'r_tibia.stl')
r_fibula = ImportVTP(vtp_path, 'r_fibula.stl')

tibia_r_transformed = tibia_r_tps.apply_spline_to_surface(r_tibia.mesh)
fibula_r_transformed = tibia_r_tps.apply_spline_to_surface(r_fibula.mesh)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_transformed_bone),  color='blue', point_size=10.0)
if len(tibia_r_exclude) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['tibia_r'].loc[tibia_r_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x246a31bef90_10&reconnect=auto" class="pyvi…

In [35]:
# import adjusted muscles and warping surfaces after control in Slicer
if os.path.isfile(os.path.join(control_path, 'new_muscles.csv')) and os.path.isfile(os.path.join(control_path, 'new_wraps.csv')):
    muscles = pd.read_csv(os.path.join(control_path, 'new_muscles.csv'),index_col='name')
    wraps = pd.read_csv(os.path.join(control_path, 'new_wraps.csv'),index_col='name')

    tibia_r_m = muscles[muscles['body'] == 'tibia_r']
    tibia_r_w = wraps[wraps['body'] == 'tibia_r']

    tibia_r_transformed_muscle = np.asarray(tibia_r_m[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')
    tibia_r_transformed_wraps[2] = np.asarray(tibia_r_w[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')

In [36]:
# create a rotation to child
bone_markers = list(mri_dict['tibia_r'].index)

tibia_r_axes = GetTibiaAxes(bone_markers = bone_markers, bone_numpy = tibia_r_transformed_bone)

# apply the rotaion
tibia_r_bone_in_child = tibia_r_axes.bone_transformed
tibia_r_skin_in_child = tibia_r_axes.apply_to_not_bone(tibia_r_transformed_skin)
tibia_r_muscles_in_child = tibia_r_axes.apply_to_not_bone(tibia_r_transformed_muscle)
tibia_r_wrap_translations_in_child = tibia_r_axes.apply_to_not_bone(tibia_r_transformed_wraps[2])

tibia_r_surface_in_child = pv.PolyData(
    tibia_r_axes.apply_to_not_bone(tibia_r_transformed.points), 
    tibia_r_transformed.faces)

fibula_r_surface_in_child = pv.PolyData(
    tibia_r_axes.apply_to_not_bone(fibula_r_transformed.points), 
    fibula_r_transformed.faces)

In [37]:
# plot results of the rotation
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['tibia_r'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['tibia_r']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(tibia_r_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(tibia_r_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x246a3262b90_11&reconnect=auto" class="pyvi…

In [38]:
# record results
surfaces = [tibia_r_surface_in_child, fibula_r_surface_in_child]
surface_names = ['tibia_r', 'fibula_r']
bone_markers = tibia_r_bone_in_child
bone_marker_names = list(mri_dict['tibia_r'].index)
skin_markers = tibia_r_skin_in_child
skin_marker_names = list(osim_skin_markers_dict['tibia_r'].index)
muscle_markers = tibia_r_muscles_in_child
muscle_marker_names = list(osim_muscls_dict['tibia_r'].index)
wrap_translations = tibia_r_wrap_translations_in_child
wrap_names = list(osim_wraps_dict['tibia_r']['name'])

pelvis_results = ScaleAndRecordData('tibia_r', output_path,
                                    transformed_bone_markers = bone_markers, bone_marker_names = bone_marker_names,
                                    transformed_skin_markers = skin_markers, skin_marker_names = skin_marker_names,
                                    transformed_muscle_markers = muscle_markers, muscle_marker_names = muscle_marker_names,
                                    transformed_wrap_translations = wrap_translations, wrap_names = wrap_names,
                                    list_transformed_surfaces_as_PolyData = surfaces, list_surface_names = surface_names )

# tibia_l  tps

In [39]:
tibia_l_exclude = []

tibia_l_tps = OneBodyTPS('tibia_l', osim_dict['tibia_l'], 
                         mri_dict['tibia_l'], 
                         osim_muscle = osim_muscls_dict['tibia_l'], 
                         osim_skin = osim_skin_markers_dict['tibia_l'],
                         exclude_bone_markers = tibia_l_exclude)

tibia_l_osim_bone = osim_dict['tibia_l']
tibia_l_transformed_bone = tibia_l_tps.transformed_bone
tibia_l_transformed_muscle = tibia_l_tps.transformed_muscle
tibia_l_transformed_skin = tibia_l_tps.transformed_skin
tibia_l_transformed_wraps = tibia_l_tps.apply_tps_to_wraps(osim_wraps_dict['tibia_l'])

l_tibia = ImportVTP(vtp_path, 'l_tibia.stl')
l_fibula = ImportVTP(vtp_path, 'l_fibula.stl')

tibia_l_transformed = tibia_l_tps.apply_spline_to_surface(l_tibia.mesh)
fibula_l_transformed = tibia_l_tps.apply_spline_to_surface(l_fibula.mesh)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(l_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_transformed_bone),  color='blue', point_size=10.0)
if len(tibia_l_exclude) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['tibia_l'].loc[tibia_l_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x246a3987f50_12&reconnect=auto" class="pyvi…

In [40]:
# #record results into the control folder
tibia_l_transformed_bone_df = pd.DataFrame(tibia_l_transformed_bone, index=tibia_l_osim_bone.index, columns=tibia_l_osim_bone.columns)
tibia_l_transformed_skin_df = pd.DataFrame(tibia_l_transformed_skin, index=osim_skin_markers_dict['tibia_l'].index, columns=osim_skin_markers_dict['tibia_l'].columns)
tibia_l_transformed_muscle_df = pd.DataFrame(tibia_l_transformed_muscle, index=osim_muscls_dict['tibia_l'].index, columns=osim_muscls_dict['tibia_l'].columns)
tibia_l_transformed_wraps_translation_df = pd.DataFrame(tibia_l_transformed_wraps[2], index=osim_wraps_dict['tibia_l']['name'], columns=['r', 'a', 's'])

tibia_l_transformed_bone_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_bone.csv'))
tibia_l_transformed_skin_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_skin.csv'))
tibia_l_transformed_muscle_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_muscle.csv'))
tibia_l_transformed_wraps_translation_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_wraps_translation.csv'))

tibia_l_transformed.save(os.path.join(control_path, f'tibia_l_transformed.stl'))
fibula_l_transformed.save(os.path.join(control_path, f'fibula_l_transformed.stl'))

In [41]:
# import adjusted muscles and warping surfaces after control in Slicer
if os.path.isfile(os.path.join(control_path, 'new_muscles.csv')) and os.path.isfile(os.path.join(control_path, 'new_wraps.csv')):
    muscles = pd.read_csv(os.path.join(control_path, 'new_muscles.csv'),index_col='name')
    wraps = pd.read_csv(os.path.join(control_path, 'new_wraps.csv'),index_col='name')

    tibia_l_m = muscles[muscles['body'] == 'tibia_l']
    tibia_l_w = wraps[wraps['body'] == 'tibia_l']

    tibia_l_transformed_muscle = np.asarray(tibia_l_m[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')
    tibia_l_transformed_wraps[2] = np.asarray(tibia_l_w[['r','a','s']].to_numpy(), dtype = np.float64, order ='C')

In [42]:
bone_markers = list(mri_dict['tibia_l'].index)

tibia_l_axes = GetTibiaAxes(bone_markers = bone_markers, bone_numpy = tibia_l_transformed_bone)
tibia_l_bone_in_child = tibia_l_axes.bone_transformed
tibia_l_skin_in_child = tibia_l_axes.apply_to_not_bone(tibia_l_transformed_skin)
tibia_l_muscles_in_child = tibia_l_axes.apply_to_not_bone(tibia_l_transformed_muscle)
tibia_l_wrap_translations_in_child = tibia_l_axes.apply_to_not_bone(tibia_l_transformed_wraps[2])

tibia_l_surface_in_child = pv.PolyData(
    tibia_l_axes.apply_to_not_bone(tibia_l_transformed.points), 
    tibia_l_transformed.faces)

fibula_l_surface_in_child = pv.PolyData(
    tibia_l_axes.apply_to_not_bone(fibula_l_transformed.points), 
    fibula_l_transformed.faces)

the determinant is less than zero, recalculate r


In [43]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(l_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['tibia_l'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['tibia_l']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(tibia_l_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(tibia_l_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53213/index.html?ui=P_0x246cd81fbd0_13&reconnect=auto" class="pyvi…

In [44]:
# record results
surfaces = [tibia_l_surface_in_child, fibula_l_surface_in_child]
surface_names = ['tibia_l', 'fibula_l']
bone_markers = tibia_l_bone_in_child
bone_marker_names = list(mri_dict['tibia_l'].index)
skin_markers = tibia_l_skin_in_child
skin_marker_names = list(osim_skin_markers_dict['tibia_l'].index)
muscle_markers = tibia_l_muscles_in_child
muscle_marker_names = list(osim_muscls_dict['tibia_l'].index)
wrap_translations = tibia_l_wrap_translations_in_child
wrap_names = list(osim_wraps_dict['tibia_l']['name'])

pelvis_results = ScaleAndRecordData('tibia_l', output_path,
                                    transformed_bone_markers = bone_markers, bone_marker_names = bone_marker_names,
                                    transformed_skin_markers = skin_markers, skin_marker_names = skin_marker_names,
                                    transformed_muscle_markers = muscle_markers, muscle_marker_names = muscle_marker_names,
                                    transformed_wrap_translations = wrap_translations, wrap_names = wrap_names,
                                    list_transformed_surfaces_as_PolyData = surfaces, list_surface_names = surface_names )